In [1]:
# Import necessary libraries
!pip install pyngrok
!pip install flask_cors
!pip install langchain_groq

from flask import Flask, request, jsonify
from pyngrok import ngrok
from flask_cors import CORS
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
import re

# Initialize the LLM
llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.1-70b-versatile"
)

# Function to parse and clean the response
def parse_and_clean_response(response):
    """Parses the raw response to extract clean questions and their options."""
    lines = response.strip().split("\n")
    clean_questions = []
    clean_options = []
    current_options = []

    for line in lines:
        if line.startswith("Q") and ":" in line:
            if current_options:
                clean_options.append(current_options)
                current_options = []
            clean_questions.append(line.strip())
        elif line.strip().startswith(("a.", "b.", "c.", "d.")):
            current_options.append(line.strip())
    if current_options:
        clean_options.append(current_options)
    return clean_questions, clean_options

# Function to generate a quiz
def generate_quiz(topic, num_questions):
    """Generates a quiz on the given topic using the LLM."""
    prompt = f"""
    You are an expert quiz generator. Create a quiz on the topic "{topic}" with {num_questions} multiple-choice questions.
    - Each question should have 4 options (a, b, c, d).
    - Provide the correct answer for each question in the format:
      Q<number>: <Question text>
      a. <Option 1>
      b. <Option 2>
      c. <Option 3>
      d. <Option 4>
      Correct answer: <a/b/c/d>
    - Ensure the questions are relevant to the topic.
    """
    messages = [HumanMessage(content=prompt)]
    response = llm.invoke(messages, max_tokens=500, temperature=0.1)
    raw_response = response.content
    questions, options = parse_and_clean_response(raw_response)
    correct_answers = []  # No answers returned by the model, so returning an empty list
    return questions, options, correct_answers

# Flask app setup
app = Flask(__name__)
CORS(app)

@app.route('/quiz', methods=['POST'])
def quiz():
    """Endpoint to generate a quiz."""
    try:
        req_data = request.get_json()
        num_questions = req_data['num_questions']
        quiz_context = req_data['quiz_context']

        questions, options, correct_answers = generate_quiz(quiz_context, num_questions)

        return jsonify({
            "questions": questions,
            "options": options,
            "correct_answers": correct_answers
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    ngrok.set_auth_token('2rA9QT8w4GXftuLHB7rLNG5R8vM_6NbtYDUYp55c4bLwthA79')  # Replace with your ngrok auth token
    public_url = ngrok.connect(5000)
    print(f"Flask app is live at {public_url}")
    app.run(port=5000)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 3.1 MB/s eta 0:00:00
Flask app is live at NgrokTunnel: "https://7438-34-30-71-244.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Jan/2025 13:14:13] "POST /quiz HTTP/1.1" 200 -
